<a href="https://colab.research.google.com/github/Sailajayadav/Chat-with-Website-Using-RAG-Pipeline/blob/main/Chat_with_Website_Using_RAG_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-cpu
from sentence_transformers import SentenceTransformer
import faiss
import requests
from bs4 import BeautifulSoup

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 42.7 MB/s eta 0:00:00


In [2]:
def crawl_and_scrape(urls):

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    content = []
    for url in urls:
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            text = ' '.join([p.get_text() for p in soup.find_all('p')])
            content.append(text)
        except Exception as e:
            print(f"Error scraping {url}: {e}")
    return ' '.join(content)

In [3]:
def chunk_text(text, chunk_size=100):

    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

In [4]:
def store_embeddings(chunks, model):

    embeddings = model.encode(chunks)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index, chunks

In [5]:
def query_vector_database(query, index, model, chunks, top_k=3):

    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    return [chunks[i] for i in indices[0]]

In [6]:
def generate_response(results):

    return "\n".join(results)

In [7]:
def main():
    # Step 1: Define URLs
    urls = [
        "https://www.uchicago.edu/",
        "https://www.washington.edu/",
        "https://www.stanford.edu/",
        "https://und.edu/",
    ]

    # Step 2: Initialize embedding model
    print("Loading embedding model...")
    model = SentenceTransformer("all-MiniLM-L6-v2")  # Pre-trained model

    # Step 3: Crawl and scrape websites
    print("Crawling and scraping websites...")
    website_data = crawl_and_scrape(urls)

    # Step 4: Chunk and embed content
    print("Chunking and embedding content...")
    chunks = chunk_text(website_data)
    index, stored_chunks = store_embeddings(chunks, model)

    # Step 5: Handle user query
    query = input("Enter your query: ")
    print("Searching content...")
    answers = query_vector_database(query, index, model, stored_chunks)
    print("Generating response...")
    response = generate_response(answers)
    print(response)

In [9]:
if __name__ == "__main__":
    main()

Loading embedding model...
Crawling and scraping websites...
Chunking and embedding content...
Enter your query: what is uchigo.edu?
Searching content...
Generating response...
A diversity of people and ideas, coupled with free and open discourse, lays the foundation for students and scholars to bring forth original ideas that define fields and enrich human life. UChicago students develop the habits of mind and intellectual skills needed to confront complex challenges. UChicago researchers have contributed to some of the world’s greatest discoveries, advancements, and bodies of knowledge. Faculty have a free and challenging environment in which to pursue the most original research. As a community partner, we invest in Chicago’s South Side across such areas as health, education, economic growth, and the arts. We are
with diverse housing, exceptional dining, and over 600 student organizations Student Affairs A rich tradition of fostering creativity and a vibrant arts district on campus S